In [2]:
import math
import hashlib

class BloomFilter:
    """
    A simple Bloom filter implementation in Python.
    """

    def __init__(self, items_count, fp_prob):
        """
        Initializes a Bloom Filter.

        Args:
            items_count (int): The expected number of items to be stored.
            fp_prob (float): The desired false positive probability.
        """
        self.fp_prob = fp_prob
        self.items_count = items_count

        # Optimal size (m) of the bit array
        self.size = self.get_size(items_count, fp_prob)

        # Optimal number of hash functions (k)
        self.hash_count = self.get_hash_count(self.size, items_count)

        # Initialize the bit array with all zeros (False)
        self.bit_array = [False] * self.size

    def __len__(self):
        """Returns the size of the bit array."""
        return self.size

    def __contains__(self, item):
        """Checks if an item is in the filter."""
        hashes = self.get_hashes(item)
        for h in hashes:
            if not self.bit_array[h]:
                return False
        return True

    def add(self, item):
        """Adds an item to the filter."""
        hashes = self.get_hashes(item)
        for h in hashes:
            self.bit_array[h] = True

    def get_hashes(self, item):
        """
        Generates k hash values for the given item.
        A single hash function (SHA256) is used to generate a digest,
        and multiple hashes are derived from it to be space-efficient.
        """
        # Convert item to bytes to be used by the hash function
        item_bytes = str(item).encode('utf-8')
        
        # Use a cryptographic hash function for good distribution
        h = hashlib.sha256(item_bytes).hexdigest()
        
        # Split the hex digest into two parts for double hashing
        h1 = int(h[:32], 16)
        h2 = int(h[32:], 16)
        
        # Generate k hash values
        return [(h1 + i * h2) % self.size for i in range(self.hash_count)]

    @staticmethod
    def get_size(n, p):
        """
        Calculates the optimal size (m) of the bit array.
        Formula: m = -(n * ln(p)) / (ln(2)^2)
        """
        m = -(n * math.log(p)) / (math.log(2) ** 2)
        return int(m)

    @staticmethod
    def get_hash_count(m, n):
        """
        Calculates the optimal number of hash functions (k).
        Formula: k = (m / n) * ln(2)
        """
        k = (m / n) * math.log(2)
        return int(k)


# --- Demo Program ---
if __name__ == "__main__":
    # 1. Define the parameters for the demo
    expected_items = 100000
    # false_positive_probability = 0.01  # Our goal is to have a 1% chance of error
    false_positive_probability = 0.001  # Our goal is to have a 0.1% chance of error

    # 2. Initialize the Bloom filter
    bloom_filter = BloomFilter(expected_items, false_positive_probability)

    print(f"Bloom Filter Demo:")
    print(f"  Expected items (n): {bloom_filter.items_count}")
    print(f"  Desired False Positive Probability (p): {bloom_filter.fp_prob}")
    print(f"  Calculated bit array size (m): {len(bloom_filter)} bits")
    print(f"  Calculated hash function count (k): {bloom_filter.hash_count}\n")

    # 3. Add a large number of items to the filter
    items_to_add = [f"user_{i}" for i in range(expected_items)]
    print(f"Adding {len(items_to_add)} items to the filter...")
    for item in items_to_add:
        bloom_filter.add(item)
    print("Items added successfully.\n")

    # 4. Test for items that definitely exist
    print("--- Testing for existing items ---")
    test_existing_items = ["user_1", "user_50000", "user_99999"]
    for item in test_existing_items:
        is_present = item in bloom_filter
        print(f"Is '{item}' in the filter? {'Yes' if is_present else 'No'} (Expected: Yes)")
    print("\n")

    # 5. Test for items that definitely DO NOT exist and count false positives
    print("--- Testing for non-existent items (checking for false positives) ---")
    false_positive_count = 0
    # Create 10,000 items that were not added to the filter
    items_to_check = [f"non_existent_user_{i}" for i in range(10000)]
    
    for item in items_to_check:
        if item in bloom_filter:
            false_positive_count += 1
            
    # 6. Display the final results
    total_checked = len(items_to_check)
    false_positive_rate = (false_positive_count / total_checked) * 100
    
    print(f"Total non-existent items checked: {total_checked}")
    print(f"Number of false positives: {false_positive_count}")
    print(f"**Actual False Positive Rate: {false_positive_rate:.2f} %**")
    print(f"**Expected False Positive Rate: {false_positive_probability * 100:.2f} %**")
    print("\nThe actual rate should be close to the expected rate.")

Bloom Filter Demo:
  Expected items (n): 100000
  Desired False Positive Probability (p): 0.001
  Calculated bit array size (m): 1437758 bits
  Calculated hash function count (k): 9

Adding 100000 items to the filter...
Items added successfully.

--- Testing for existing items ---
Is 'user_1' in the filter? Yes (Expected: Yes)
Is 'user_50000' in the filter? Yes (Expected: Yes)
Is 'user_99999' in the filter? Yes (Expected: Yes)


--- Testing for non-existent items (checking for false positives) ---
Total non-existent items checked: 10000
Number of false positives: 8
**Actual False Positive Rate: 0.08 %**
**Expected False Positive Rate: 0.10 %**

The actual rate should be close to the expected rate.
